<a href="https://colab.research.google.com/github/sy178sy/sy178sy.github.io/blob/master/Gartner_Hackelite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from inspect import signature


In [0]:
from google.colab import files
uploaded = files.upload()


In [0]:
import io
train=pd.read_csv(io.BytesIO(uploaded['Train.csv']))
test=pd.read_csv(io.BytesIO(uploaded['Test.csv']))
train.head(5)

,Client ID,Company ID,Client Contract Starting Month,Flag 1,Flag 2,Flag 3,Flag 4,Flag 5,Flag 6,Activity 1 Time Period 11,Activity 1 Time Period 10,Activity 1 Time Period 9,Activity 1 Time Period 8,Activity 1 Time Period 7,Activity 1 Time Period 6,Activity 1 Time Period 5,Activity 1 Time Period 4,Activity 1 Time Period 3,Activity 1 Time Period 2,Activity 1 Time Period 1,Activity 1 Time Period 0,Activity 2 Time Period 11,Activity 2 Time Period 10,Activity 2 Time Period 9,Activity 2 Time Period 8,Activity 2 Time Period 7,Activity 2 Time Period 6,Activity 2 Time Period 5,Activity 2 Time Period 4,Activity 2 Time Period 3,Activity 2 Time Period 2,Activity 2 Time Period 1,Activity 2 Time Period 0,Activity 3 Time Period 11,Activity 3 Time Period 10,Activity 3 Time Period 9,Activity 3 Time Period 8,Activity 3 Time Period 7,Activity 3 Time Period 6,Activity 3 Time Period 5,...,Activity 5 Time Period 2,Activity 5 Time Period 1,Activity 5 Time Period 0,Activity 6 Time Period 11,Activity 6 Time Period 10,Activity 6 Time Period 9,Activity 6 Time Period 8,Activity 6 Time Period 7,Activity 6 Time Period 6,Activity 6 Time Period 5,Activity 6 Time Period 4,Activity 6 Time Period 3,Activity 6 Time Period 2,Activity 6 Time Period 1,Activity 6 Time Period 0,Activity 7 Time Period 11,Activity 7 Time Period 10,Activity 7 Time Period 9,Activity 7 Time Period 8,Activity 7 Time Period 7,Activity 7 Time Period 6,Activity 7 Time Period 5,Activity 7 Time Period 4,Activity 7 Time Period 3,Activity 7 Time Period 2,Activity 7 Time Period 1,Activity 7 Time Period 0,Activity 8 Time Period 11,Activity 8 Time Period 10,Activity 8 Time Period 9,Activity 8 Time Period 8,Activity 8 Time Period 7,Activity 8 Time Period 6,Activity 8 Time Period 5,Activity 8 Time Period 4,Activity 8 Time Period 3,Activity 8 Time Period 2,Activity 8 Time Period 1,Activity 8 Time Period 0,Client Retention Flag
0,10004519380,6294937,Jun,Multi Year,2,Yes,Unknown,8,Manufacturing,24,16,20,52,0,14,94,44,16,22,10,40,2,0,0,0,0,1,27,13,4,0,0,3,0,2,1,1,1,1,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,1,0,Yes
1,10004521476,6293796,Mar,Multi Year,7,No,Yes,6,Government,9,2,0,18,27,0,26,2,6,6,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,No
2,10004515403,6293911,Dec,Multi Year,2,No,Unknown,15,Technology and Telecom,1,11,12,0,11,13,5,6,11,38,4,1,0,1,0,2,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Yes
3,10004511478,6293048,Feb,Multi Year,2,No,Unknown,15,Technology and Telecom,1,0,0,0,2,0,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,No
4,10004516858,6294364,Jun,Multi Year,7,No,Unknown,9,"Banking, Finance & Ins",16,14,48,6,6,0,8,18,12,34,40,12,9,0,0,0,0,0,0,1,0,5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Yes


In [0]:
def data_cleaning(train):

    train.loc[train["Flag 1"]=="Single Year","Flag 1"]=0
    train.loc[train["Flag 1"]=="Multi Year","Flag 1"]=1
    train.loc[train["Flag 6"]=="Manufacturing","Flag 6"]=1
    train.loc[train["Flag 6"]=="Government","Flag 6"]=2
    train.loc[train["Flag 6"]=="Technology and Telecom","Flag 6"]=3
    train.loc[train["Flag 6"]=="Banking, Finance & Ins","Flag 6"]=4
    train.loc[train["Flag 6"]=="Education","Flag 6"]=5
    train.loc[train["Flag 6"]=="Energy & Utilities","Flag 6"]=6
    train.loc[train["Flag 6"]=="Healthcare","Flag 6"]=10
    
    train.loc[train["Flag 6"]=="Services","Flag 6"]=7
    train.loc[train["Flag 6"]=="Retail","Flag 6"]=8
    train.loc[train["Flag 6"]=="Unknown","Flag 6"]=9
  
    
    return train

In [0]:
train=data_cleaning(train)
test=data_cleaning(test)
train.head(10)


,Client ID,Company ID,Client Contract Starting Month,Flag 1,Flag 2,Flag 3,Flag 4,Flag 5,Flag 6,Activity 1 Time Period 11,Activity 1 Time Period 10,Activity 1 Time Period 9,Activity 1 Time Period 8,Activity 1 Time Period 7,Activity 1 Time Period 6,Activity 1 Time Period 5,Activity 1 Time Period 4,Activity 1 Time Period 3,Activity 1 Time Period 2,Activity 1 Time Period 1,Activity 1 Time Period 0,Activity 2 Time Period 11,Activity 2 Time Period 10,Activity 2 Time Period 9,Activity 2 Time Period 8,Activity 2 Time Period 7,Activity 2 Time Period 6,Activity 2 Time Period 5,Activity 2 Time Period 4,Activity 2 Time Period 3,Activity 2 Time Period 2,Activity 2 Time Period 1,Activity 2 Time Period 0,Activity 3 Time Period 11,Activity 3 Time Period 10,Activity 3 Time Period 9,Activity 3 Time Period 8,Activity 3 Time Period 7,Activity 3 Time Period 6,Activity 3 Time Period 5,...,Activity 5 Time Period 2,Activity 5 Time Period 1,Activity 5 Time Period 0,Activity 6 Time Period 11,Activity 6 Time Period 10,Activity 6 Time Period 9,Activity 6 Time Period 8,Activity 6 Time Period 7,Activity 6 Time Period 6,Activity 6 Time Period 5,Activity 6 Time Period 4,Activity 6 Time Period 3,Activity 6 Time Period 2,Activity 6 Time Period 1,Activity 6 Time Period 0,Activity 7 Time Period 11,Activity 7 Time Period 10,Activity 7 Time Period 9,Activity 7 Time Period 8,Activity 7 Time Period 7,Activity 7 Time Period 6,Activity 7 Time Period 5,Activity 7 Time Period 4,Activity 7 Time Period 3,Activity 7 Time Period 2,Activity 7 Time Period 1,Activity 7 Time Period 0,Activity 8 Time Period 11,Activity 8 Time Period 10,Activity 8 Time Period 9,Activity 8 Time Period 8,Activity 8 Time Period 7,Activity 8 Time Period 6,Activity 8 Time Period 5,Activity 8 Time Period 4,Activity 8 Time Period 3,Activity 8 Time Period 2,Activity 8 Time Period 1,Activity 8 Time Period 0,Client Retention Flag
0,10004519380,6294937,Jun,1,2,Yes,Unknown,8,1,24,16,20,52,0,14,94,44,16,22,10,40,2,0,0,0,0,1,27,13,4,0,0,3,0,2,1,1,1,1,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,1,0,Yes
1,10004521476,6293796,Mar,1,7,No,Yes,6,2,9,2,0,18,27,0,26,2,6,6,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,No
2,10004515403,6293911,Dec,1,2,No,Unknown,15,3,1,11,12,0,11,13,5,6,11,38,4,1,0,1,0,2,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Yes
3,10004511478,6293048,Feb,1,2,No,Unknown,15,3,1,0,0,0,2,0,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,No
4,10004516858,6294364,Jun,1,7,No,Unknown,9,4,16,14,48,6,6,0,8,18,12,34,40,12,9,0,0,0,0,0,0,1,0,5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Yes
5,10004521519,6292276,Jun,0,7,No,Yes,3,5,10,6,30,52,0,0,36,0,0,56,0,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,Yes
6,10004510565,6294388,May,1,2,No,Unknown,16,2,0,0,2,0,2,0,4,0,0,0,20,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Yes
7,10004519510,6292068,Jul,0,2,Yes,Unknown,3,5,0,6,14,0,2,2,0,0,0,4,0,10,0,0,0,0,2,0,0,0,0,0,0,1,2,2,0,3,2,0,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Yes
8,10004521134,6293710,Oct,1,2,Yes,Yes,7,2,14,28,22,38,16,6,0,0,8,0,4,12,0,0,0,4,0,2,0,0,1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,Yes
9,10004519931,6293762,May,1,2,Yes,Yes,3,5,64,144,212,220,96,98,22,26,0,2,2,92,0,1,1,2,0,0,0,0,0,0,0,7,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Yes


In [0]:
train['Frequency of Act1 and Act2']=train['Activity 1 Time Period 0']+train['Activity 1 Time Period 1']+train['Activity 1 Time Period 2']+train['Activity 1 Time Period 3']+train['Activity 1 Time Period 4']+train['Activity 1 Time Period 5']+train['Activity 1 Time Period 6']+train['Activity 1 Time Period 7']+train['Activity 1 Time Period 8']+train['Activity 1 Time Period 9']+train['Activity 1 Time Period 10']+train['Activity 1 Time Period 11']+train['Activity 2 Time Period 0']+train['Activity 2 Time Period 1']+train['Activity 2 Time Period 2']+train['Activity 2 Time Period 3']+train['Activity 2 Time Period 4']+train['Activity 2 Time Period 5']+train['Activity 2 Time Period 6']+train['Activity 2 Time Period 7']+train['Activity 2 Time Period 8']+train['Activity 2 Time Period 9']+train['Activity 2 Time Period 10']+train['Activity 2 Time Period 11']
train['Frequency of other Act']=train['Activity 3 Time Period 0']+train['Activity 3 Time Period 1']+train['Activity 3 Time Period 2']+train['Activity 3 Time Period 3']+train['Activity 3 Time Period 4']+train['Activity 3 Time Period 5']+train['Activity 3 Time Period 6']+train['Activity 3 Time Period 7']+train['Activity 3 Time Period 8']+train['Activity 3 Time Period 9']+train['Activity 3 Time Period 10']+train['Activity 3 Time Period 11']+train['Activity 4 Time Period 0']+train['Activity 4 Time Period 1']+train['Activity 4 Time Period 2']+train['Activity 4 Time Period 3']+train['Activity 4 Time Period 4']+train['Activity 4 Time Period 5']+train['Activity 4 Time Period 6']+train['Activity 4 Time Period 7']+train['Activity 4 Time Period 8']+train['Activity 4 Time Period 9']+train['Activity 4 Time Period 10']+train['Activity 4 Time Period 11']+train['Activity 8 Time Period 0']+train['Activity 8 Time Period 1']+train['Activity 8 Time Period 2']+train['Activity 8 Time Period 3']+train['Activity 8 Time Period 4']+train['Activity 8 Time Period 5']+train['Activity 8 Time Period 6']+train['Activity 8 Time Period 7']+train['Activity 8 Time Period 8']+train['Activity 8 Time Period 9']+train['Activity 8 Time Period 10']+train['Activity 8 Time Period 11']
train['transformlog_freq of Act1 and Act2']=np.log(train['Frequency of Act1 and Act2'])

In [0]:
test['Frequency of Act1 and Act2']=test['Activity 2 Time Period 0']+test['Activity 2 Time Period 1']+test['Activity 2 Time Period 2']+test['Activity 2 Time Period 3']+test['Activity 2 Time Period 4']+test['Activity 2 Time Period 5']+test['Activity 2 Time Period 6']+test['Activity 2 Time Period 7']+test['Activity 2 Time Period 8']+test['Activity 2 Time Period 9']+test['Activity 2 Time Period 10']+test['Activity 2 Time Period 11']+test['Activity 1 Time Period 0']+test['Activity 1 Time Period 1']+test['Activity 1 Time Period 2']+test['Activity 1 Time Period 3']+test['Activity 1 Time Period 4']+test['Activity 1 Time Period 5']+test['Activity 1 Time Period 6']+test['Activity 1 Time Period 7']+test['Activity 1 Time Period 8']+test['Activity 1 Time Period 9']+test['Activity 1 Time Period 10']+test['Activity 1 Time Period 11']
test['Frequency of other Act']=test['Activity 3 Time Period 0']+test['Activity 3 Time Period 1']+test['Activity 3 Time Period 2']+test['Activity 3 Time Period 3']+test['Activity 3 Time Period 4']+test['Activity 3 Time Period 5']+test['Activity 3 Time Period 6']+test['Activity 3 Time Period 7']+test['Activity 3 Time Period 8']+test['Activity 3 Time Period 9']+test['Activity 3 Time Period 10']+test['Activity 3 Time Period 11']+test['Activity 4 Time Period 0']+test['Activity 4 Time Period 1']+test['Activity 4 Time Period 2']+test['Activity 4 Time Period 3']+test['Activity 4 Time Period 4']+test['Activity 4 Time Period 5']+test['Activity 4 Time Period 6']+test['Activity 4 Time Period 7']+test['Activity 4 Time Period 8']+test['Activity 4 Time Period 9']+test['Activity 4 Time Period 10']+test['Activity 4 Time Period 11']+test['Activity 8 Time Period 0']+test['Activity 8 Time Period 1']+test['Activity 8 Time Period 2']+test['Activity 8 Time Period 3']+test['Activity 8 Time Period 4']+test['Activity 8 Time Period 5']+test['Activity 8 Time Period 6']+test['Activity 8 Time Period 7']+test['Activity 8 Time Period 8']+test['Activity 8 Time Period 9']+test['Activity 8 Time Period 10']+test['Activity 8 Time Period 11']
train['transformlog_freq of Act1 and Act2']=np.log(train['Frequency of Act1 and Act2'])

test.head(6)

,Client ID,Company ID,Client Contract Starting Month,Flag 1,Flag 2,Flag 3,Flag 4,Flag 5,Flag 6,Activity 1 Time Period 11,Activity 1 Time Period 10,Activity 1 Time Period 9,Activity 1 Time Period 8,Activity 1 Time Period 7,Activity 1 Time Period 6,Activity 1 Time Period 5,Activity 1 Time Period 4,Activity 1 Time Period 3,Activity 1 Time Period 2,Activity 1 Time Period 1,Activity 1 Time Period 0,Activity 2 Time Period 11,Activity 2 Time Period 10,Activity 2 Time Period 9,Activity 2 Time Period 8,Activity 2 Time Period 7,Activity 2 Time Period 6,Activity 2 Time Period 5,Activity 2 Time Period 4,Activity 2 Time Period 3,Activity 2 Time Period 2,Activity 2 Time Period 1,Activity 2 Time Period 0,Activity 3 Time Period 11,Activity 3 Time Period 10,Activity 3 Time Period 9,Activity 3 Time Period 8,Activity 3 Time Period 7,Activity 3 Time Period 6,Activity 3 Time Period 5,...,Activity 5 Time Period 0,Activity 6 Time Period 11,Activity 6 Time Period 10,Activity 6 Time Period 9,Activity 6 Time Period 8,Activity 6 Time Period 7,Activity 6 Time Period 6,Activity 6 Time Period 5,Activity 6 Time Period 4,Activity 6 Time Period 3,Activity 6 Time Period 2,Activity 6 Time Period 1,Activity 6 Time Period 0,Activity 7 Time Period 11,Activity 7 Time Period 10,Activity 7 Time Period 9,Activity 7 Time Period 8,Activity 7 Time Period 7,Activity 7 Time Period 6,Activity 7 Time Period 5,Activity 7 Time Period 4,Activity 7 Time Period 3,Activity 7 Time Period 2,Activity 7 Time Period 1,Activity 7 Time Period 0,Activity 8 Time Period 11,Activity 8 Time Period 10,Activity 8 Time Period 9,Activity 8 Time Period 8,Activity 8 Time Period 7,Activity 8 Time Period 6,Activity 8 Time Period 5,Activity 8 Time Period 4,Activity 8 Time Period 3,Activity 8 Time Period 2,Activity 8 Time Period 1,Activity 8 Time Period 0,Frequency of Act1 and Act2,Frequency of other Act,transformlog_freq of Act1 and Act2
0,10004520718,6295189,Apr,0,10,Yes,Yes,8,1,47,37,21,68,2,6,15,37,16,26,11,11,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,299,3,1.098612
1,10004519187,6295450,May,0,2,Yes,Unknown,3,5,116,14,18,40,70,30,50,64,58,54,48,32,14,0,0,6,5,0,0,1,0,3,17,2,2,1,0,2,3,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,642,21,3.044522
2,10004515634,6292422,Jun,0,2,Yes,Unknown,3,5,0,18,0,14,10,0,4,42,18,12,4,12,0,0,0,1,0,0,0,5,2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,142,3,1.098612
3,10004512321,6292318,Jul,0,2,No,Unknown,15,3,0,0,0,0,0,2,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,2,0.693147
4,10004518550,6294996,Jun,0,10,No,Unknown,8,1,0,14,26,72,146,26,32,82,120,58,98,98,0,3,1,0,2,2,0,3,1,3,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,789,1,0.000000
5,10004522099,6293039,Jun,1,6,Yes,Yes,8,4,22,0,16,4,8,4,8,20,6,14,32,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,148,10,2.302585


In [0]:

def datacleaning(train):
    train.loc[train["transformlog_freq of Act1 and Act2"]=="-inf","transformlog_freq of Act1 and Act2"]=0
    return train
  
  

  
    

In [0]:
train=datacleaning(train)
test=datacleaning(test) 

test

In [0]:
predictor_vars=['transformlog_freq of Act1 and Act2','Frequency of other Act']

predictor_vars



In [0]:
pred=["Client Retention Flag"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train[predictor_vars],train[pred],test_size=0.2, random_state=42)

In [0]:
Model=RandomForestClassifier(max_depth=11)
                             

In [0]:
Model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=11, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators='warn',
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:

Pred=Model.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [0]:
y_pred = Model.predict(X_test)

In [0]:

precision_score(y_test,y_pred,pos_label='Yes')

0.7387566137566137

In [0]:
recall_score(y_test, y_pred,pos_label='Yes')

0.9059205190592052

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7086659064994298

In [0]:
c

0.8138433515482696

In [0]:
X_test1=test[predictor_vars]

pred_1=Model.predict(X_test1)

In [0]:
df=pd.DataFrame(pred_1)

In [0]:

df.to_csv(r'C:\Users\HP\Desktop\Book.csv')

In [0]:
df

In [0]:
from google.colab import drive
drive.mount('drive')

In [0]:
df.to_csv('Book.csv')
!cp Book.csv drive/My\ Drive/